### Task 1. 

#### MapReduce

Use MapReduce to calculate the average housing price in each zip code and filter out records in which gross sqft is less than 500 and sale price is less than 100000. Round the average housing price to 2 decimals, and sort the output by housing price ascending.

In [1]:
from functools import reduce
import pandas as pd
import csv
from functools import reduce
from multiprocessing import Pool

In [2]:
!wget -O housingSalseSample.csv https://github.com/CUSP2021ADS/Data/raw/main/housingSalseSample.csv

--2023-03-14 22:00:52--  https://github.com/CUSP2021ADS/Data/raw/main/housingSalseSample.csv
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/CUSP2021ADS/Data/main/housingSalseSample.csv [following]
--2023-03-14 22:00:52--  https://raw.githubusercontent.com/CUSP2021ADS/Data/main/housingSalseSample.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6167298 (5.9M) [text/plain]
Saving to: 'housingSalseSample.csv'

housingSalseSample. 100%[===================>]   5.88M  1.37MB/s    in 4.3s    

2023-03-14 22:00:57 (1.37 MB/s) - 'housingSalseSample.csv' saved [6167298/6167298]



In [3]:
house = pd.read_csv('housingSalseSample.csv')
house

,BOROUGH,BLOCK,LOT,BUILDING CLASS CATEGORY,ADDRESS,ZIP CODE,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE,SALE DATE
0,1,1170,1010,17 CONDOPS,"200 WEST 79TH STREET, 7P",10024.0,0.0,1975.0,551000,2011-07-28
1,4,1275,45,10 COOPS - ELEVATOR APARTMENTS,"35-35 75TH STREET, 304",11372.0,0.0,1941.0,338000,2011-05-20
2,3,5746,58,03 THREE FAMILY DWELLINGS,1249 65TH STREET,11219.0,3861.0,1910.0,1250000,2014-12-04
3,4,2086,50,10 COOPS - ELEVATOR APARTMENTS,"97-07 63 ROAD, 8F",11374.0,0.0,1955.0,253000,2016-03-09
4,4,5028,7,10 COOPS - ELEVATOR APARTMENTS,"38-15 149TH STREET, 1F",11354.0,0.0,1955.0,0,2016-12-30
...,...,...,...,...,...,...,...,...,...,...
49995,5,143,66,01 ONE FAMILY DWELLINGS,29 NORTH MADA AVENUE,10310.0,1740.0,1915.0,0,2014-10-15
49996,4,4686,34,02 TWO FAMILY DWELLINGS,150-38 17TH ROAD,11357.0,2945.0,1925.0,970000,2014-10-28
49997,4,6670,1,09 COOPS - WALKUP APARTMENTS,"144-32 78TH AVENUE, 2F",11367.0,0.0,1949.0,140000,2014-10-17
49998,1,1247,37,10 COOPS - ELEVATOR APARTMENTS,"302 WEST 86TH STREET, 1A",10024.0,0.0,1917.0,880000,2012-10-31


In [4]:
house.describe()

,BOROUGH,BLOCK,LOT,ZIP CODE,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE
count,50000.000000,50000.000000,50000.000000,50000.00000,4.887300e+04,49621.000000,5.000000e+04
mean,2.878360,4010.394140,380.752600,10770.75998,4.240113e+03,1769.322605,9.679545e+05
std,1.325413,3523.195821,661.725832,889.28757,2.572600e+04,565.616599,6.405353e+06
min,1.000000,1.000000,1.000000,0.00000,0.000000e+00,0.000000,0.000000e+00
25%,1.000000,1186.000000,22.000000,10128.00000,0.000000e+00,1920.000000,0.000000e+00
50%,3.000000,2849.000000,50.000000,11206.00000,1.185000e+03,1935.000000,3.400000e+05
75%,4.000000,5953.000000,1002.000000,11355.00000,2.280000e+03,1965.000000,7.350000e+05
max,5.000000,16350.000000,9117.000000,11697.00000,1.743584e+06,2019.000000,4.512500e+08


In [5]:
def returnInfo(record): #extract necessary fields from a data record (raw)
    return(record['ZIP CODE'], record['GROSS SQUARE FEET'], record['SALE PRICE'])

#filter the price and square feet
def filterHouse(record): 
    if (record[1] != '') :
        square_feet = float(record[1])
        price = float(record[2])        
        if (price >= 100000) & (square_feet >= 500):
            return True
        else:
            return False
#cal the price for each row
def AvePrice(record): 
    zipcode = record[0]
    square_feet = float(record[1])
    price = float(record[2])
    return [zipcode, price / square_feet]

#get the zipcode price list
def Zip_price(result,record):
    zipcode = record[0]
    ave_Price = float(record[1])
    result[zipcode] = result.get(zipcode,[]) + [ave_Price]
    return result    

#get the average price for each zipcode and round
def List_Price(record):
    zipcode = record[0]
    ave_Price = record[1]
    price = [zipcode,round(sum(ave_Price)/len(ave_Price),2),]
    return price


In [6]:
with open('./housingSalseSample.csv') as fi:
    reader = csv.DictReader(fi) 
    output = sorted(map(List_Price, 
                      reduce(Zip_price, 
                                 map(AvePrice, 
                                     filter(filterHouse, 
                                         map(returnInfo,reader)
                                     ),
                                    ),
                             {}).items()),key=lambda x: x[1])


In [7]:
output

[['10474.0', 147.17],
 ['10457.0', 165.29],
 ['10040.0', 165.67],
 ['10453.0', 179.31],
 ['10460.0', 181.66],
 ['10456.0', 188.21],
 ['10459.0', 193.89],
 ['10455.0', 196.87],
 ['10458.0', 202.23],
 ['10473.0', 204.08],
 ['10454.0', 207.71],
 ['10470.0', 211.52],
 ['10472.0', 214.66],
 ['10467.0', 215.39],
 ['10452.0', 215.58],
 ['11239.0', 217.06],
 ['11692.0', 221.92],
 ['10466.0', 224.77],
 ['10468.0', 225.43],
 ['11691.0', 225.84],
 ['10303.0', 232.12],
 ['10462.0', 233.26],
 ['10469.0', 234.05],
 ['11208.0', 236.5],
 ['11207.0', 237.93],
 ['11203.0', 243.22],
 ['10302.0', 253.71],
 ['10465.0', 255.11],
 ['10461.0', 255.11],
 ['10475.0', 259.59],
 ['11212.0', 261.47],
 ['11233.0', 262.29],
 ['11418.0', 262.78],
 ['11429.0', 266.09],
 ['11693.0', 266.45],
 ['10310.0', 269.86],
 ['11434.0', 269.93],
 ['11413.0', 271.72],
 ['11412.0', 272.91],
 ['11433.0', 273.22],
 ['10307.0', 275.99],
 ['10301.0', 276.86],
 ['11236.0', 277.42],
 ['10034.0', 279.01],
 ['11416.0', 282.99],
 ['10309.0'

In [8]:
'''grading function
please pass your output into this function and upload the generated csv file 
together with your notebook to your GitHub repo
50 pts, no partial credits'''

def grading(output):
    import pandas as pd
    pd.DataFrame(output,columns=['zipcode','price']).to_csv('output1.csv',index=False)

grading(output)

### Task 2. 
#### multiprocessing and Chunk

In [9]:
!wget -O ADSSession6Task2Data.csv https://raw.githubusercontent.com/CUSP2021ADS/Data/main/ADSSession6Task2Data.csv

--2023-03-14 22:00:57--  https://raw.githubusercontent.com/CUSP2021ADS/Data/main/ADSSession6Task2Data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 279369 (273K) [text/plain]
Saving to: 'ADSSession6Task2Data.csv'

ADSSession6Task2Dat 100%[===================>] 272.82K  1.76MB/s    in 0.2s    

2023-03-14 22:00:57 (1.76 MB/s) - 'ADSSession6Task2Data.csv' saved [279369/279369]



Apply the chunk and MapReduce method to sum up each type's value in the synthetic dataset—only keep records which $Select$ value is 1, and read 1000 lines in each chunk. Use multiprocessing package to deploy the task to multi-cores. The output should only have two columns: type and total value.

Hint: filter function is not available in multiprocessing, implement an if-else function in the map or reduce step.

In [10]:
data_task2=pd.read_csv('ADSSession6Task2Data.csv')
data_task2

,Type,Value,Select
0,0,0.1488486295446153,0
1,3,0.2198529412734188,1
2,7,0.15871379686112896,0
3,0,0.9980337347337275,1
4,3,0.4923200916998063,0
...,...,...,...
11997,8,0.9252856634364992,0
11998,6,0.12564358624103245,1
11999,7,0.9328986651807677,0
12000,2,0.2858165168913036,0


In [11]:
#due to the error, we write our function from another ipynb
from multiprocessing import Pool
from ipynb.fs.full.def_func import returnInfo
from ipynb.fs.full.def_func import filterData
from ipynb.fs.full.def_func import sum_Value

In [12]:
%%time
with open('./ADSSession6Task2Data.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    chunk = []
    result = []
    chunksize = 1000
    pool = Pool(8)
    for i, line in enumerate(reader):
        if (i % chunksize == 0 and i > 0):
            output = list(reduce(sum_Value, 
                                 pool.map(filterData,
                                          pool.map(returnInfo,chunk)),{}).items())
            result += output
            chunk = []
        chunk.append(line)       
    output = list(reduce(sum_Value, 
                         pool.map(filterData,
                                  pool.map(returnInfo,chunk)),
                         {}).items())
    result += output
    pool.close()

CPU times: user 75.6 ms, sys: 115 ms, total: 190 ms
Wall time: 748 ms


In [13]:
output = list(reduce(sum_Value,result,{}).items())

In [14]:
'''grading function
please pass your output into this function and upload the generated csv file 
together with your notebook to your GitHub repo
50 pts, no partial credits'''

def grading(output):
    import pandas as pd
    pd.DataFrame(output,columns=['Type','Value']).to_csv('output2.csv',index=False)

grading(output)